###  Retrieval-Augmented Generation (RAG) for Inventory Scoring

#### Import Packages

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from rag.rag_pipeline import Rag
import pandas as pd

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-22 12:56:31,847 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-22 12:56:31,852 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'
2024-03-22 12:56:31,873 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-22 12:56:31,876 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'


In [3]:
rag_chain = Rag()

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-22 12:56:36,169 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-03-22 12:56:36,172 - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2024-03-22 12:56:37,253 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-22 12:56:37,481 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-03-22 12:56:37,742 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 200 0
2024-03-22 12:56:38,175 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all

#### Populate Merged Data into vector database

In [4]:
merged_df = pd.read_csv("../data/merged_data.csv")

2024-03-22 12:56:40,679 - DEBUG - Starting new HTTPS connection (1): us-api.i.posthog.com:443
2024-03-22 12:56:42,209 - DEBUG - https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 None


In [7]:
merged_df.head()

,Unnamed: 0,campaign_id,budget_amount,budget_currencycode,KPI,pricing_model,geo_targeting,vertical,targeting,game_key,...,devicetype,devicemake,site_name,matchedfoldposition,browser,impression,engagement,click,CTR,ER
0,0,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Tablet,Samsung,www.yahoo.com,Unknown,Chrome,57.0,0.0,0.0,0.000000,0.000000
1,1,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Apple,www.yahoo.com,Unknown,Mobile Safari,983.0,265.0,71.0,7.222787,26.958291
2,2,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Samsung,www.yahoo.com,Unknown,Chrome,9.0,0.0,0.0,0.000000,0.000000
3,3,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Apple,www.the-sun.com,Unknown,Mobile Safari UI/WKWebView,3.0,0.0,0.0,0.000000,0.000000
4,4,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Samsung,currently.att.yahoo.com,Unknown,Chrome Mobile,27.0,3.0,0.0,0.000000,11.111111


In [5]:
merged_df = merged_df.drop(columns=['Unnamed: 0', 'impression', 'engagement', 'click', 'budget_currencycode', 'budget_amount', 'pricing_model'])

In [6]:
sample_data = merged_df.iloc[:10, :].to_dict(orient='records')

In [92]:
sample_data

[{'campaign_id': 'f4p5a01',
  'KPI': 'Engagement',
  'geo_targeting': 'All Country',
  'vertical': 'Sports',
  'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
  'game_key': 'b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed3176d99',
  'adformat': '320x480',
  'creative': 'Sensory Video',
  'renderingcontext': 'MobileOptimizedWeb',
  'osfamily': 'Android',
  'devicetype': 'Tablet',
  'devicemake': 'Samsung',
  'site_name': 'www.yahoo.com',
  'matchedfoldposition': 'Unknown',
  'browser': 'Chrome',
  'CTR': 0.0,
  'ER': 0.0},
 {'campaign_id': 'f4p5a01',
  'KPI': 'Engagement',
  'geo_targeting': 'All Country',
  'vertical': 'Sports',
  'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
  'game_key': 'b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed3176d99',
  'adformat': '320x480',
  'creative': 'Sensory Video',
  'renderingcontext': 'MobileOptimizedWeb',
  'osfamily': 'iOS',
  'devicetype': 'Mobile',
  'devicemake': 'Apple',
  'si

In [89]:
# for key, value in sample_data[0].items():
#     print(f"column: {key}   data type: {type(value)}")
sample_data = [{
  'campaign_id': 'f4p5a01',
  'KPI': 'Engagement',
  'geo_targeting': 'All Country',
  'vertical': 'Sports',
  'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
  'game_key': 'b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed3176d99',
  'adformat': '320x480',
  'creative': 'Sensory Video',
  'renderingcontext': 'MobileOptimizedWeb',
  'osfamily': 'iOS',
  'devicetype': 'Mobile',
  'devicemake': 'Apple',
  'site_name': 'www.yahoo.com',
  'matchedfoldposition': 'Unknown',
  'browser': 'Chrome Mobile iOS',
  'CTR': 8.2417582,
  'ER': 22.527472}]

#### Create Weivate Database

In [53]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import os

# Replace with your Weaviate connection details
client = weaviate.Client( embedded_options = EmbeddedOptions(),
                           additional_headers = {
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY") # Replace with your inference API key
    })

2024-03-22 11:15:22,681 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:15:22,687 - DEBUG - http://localhost:8079 "GET /v1/.well-known/ready HTTP/1.1" 200 0
2024-03-22 11:15:22,695 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:15:22,698 - DEBUG - http://localhost:8079 "GET /v1/.well-known/openid-configuration HTTP/1.1" 404 0
2024-03-22 11:15:22,704 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:15:22,706 - DEBUG - http://localhost:8079 "GET /v1/meta HTTP/1.1" 200 871
2024-03-22 11:15:22,711 - DEBUG - Starting new HTTPS connection (1): pypi.org:443


embedded weaviate is already listening on port 8079


2024-03-22 11:15:23,122 - DEBUG - https://pypi.org:443 "GET /pypi/weaviate-client/json HTTP/1.1" 200 49836
/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [87]:
# client.schema.delete_class("inventory_data")

2024-03-22 11:22:25,001 - DEBUG - Resetting dropped connection: localhost
2024-03-22 11:22:25,161 - DEBUG - http://localhost:8079 "DELETE /v1/schema/Inventory_data HTTP/1.1" 200 0


In [88]:
client.schema.create("../db/schema.json")

{"level":"info","msg":"Created shard inventory_data_vluz58ZF59RZ in 2.183838ms","time":"2024-03-22T11:22:31+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T11:22:31+03:00","took":117848}
2024-03-22 11:22:31,885 - DEBUG - http://localhost:8079 "POST /v1/schema HTTP/1.1" 200 None


In [94]:
client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, data in enumerate(sample_data):  # Batch import data
        print(i+1)
        response = batch.add_data_object(
            data_object=data,
            class_name="inventory_data"
        )
        print(response)
    print("done")


1
07c8704a-783a-4381-8857-8e61d232f1cb
2
cf58c0a1-7c54-49bc-be05-ecff32c6fd98
3
de3c10c4-0792-47d9-a7b6-78db9ae342c1
4
10ee1420-287a-4a3e-a051-3a173186fb18
5
40f800b7-c476-4b81-88f7-8107d243c13c
6
659ae2f8-8049-479a-826f-9e12046a180d
7
9dcb04a6-a3a5-4701-ac4d-638adb3b6610
8
3eeb6e36-dcfe-4abe-9d49-d8a3fb6bcc1d
9
91a34113-2ab3-4ac0-a34f-d12ba6d147be
10
3a9f4b1e-a61f-4ff1-876c-3fe33c0a8cb4
done


InvalidJSONError: Out of range float values are not JSON compliant

In [25]:
merged_df.head()

,campaign_id,KPI,geo_targeting,vertical,targeting,game_key,adformat,creative,renderingcontext,osfamily,devicetype,devicemake,site_name,matchedfoldposition,browser,CTR,ER
0,f4p5a01,Engagement,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,Sensory Video,MobileOptimizedWeb,Android,Tablet,Samsung,www.yahoo.com,Unknown,Chrome,0.000000,0.000000
1,f4p5a01,Engagement,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,Sensory Video,MobileOptimizedWeb,iOS,Mobile,Apple,www.yahoo.com,Unknown,Mobile Safari,7.222787,26.958291
2,f4p5a01,Engagement,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,Sensory Video,MobileOptimizedWeb,Android,Mobile,Samsung,www.yahoo.com,Unknown,Chrome,0.000000,0.000000
3,f4p5a01,Engagement,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,Sensory Video,MobileOptimizedWeb,iOS,Mobile,Apple,www.the-sun.com,Unknown,Mobile Safari UI/WKWebView,0.000000,0.000000
4,f4p5a01,Engagement,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,Sensory Video,MobileOptimizedWeb,Android,Mobile,Samsung,currently.att.yahoo.com,Unknown,Chrome Mobile,0.000000,11.111111


In [136]:
import json

response = (
    client.query
    .get("inventory_data", ["campaign_id", "creative", "devicetype"])
    .with_near_text({"concepts": ["Arts & Entertainment"]})
    .with_near_vector
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=4))

2024-03-22 06:05:17,445 - DEBUG - Resetting dropped connection: localhost
{"action":"requests_total","api":"graphql","class_name":"","error":"Unknown argument \"nearText\" on field \"Inventory_data\" of type \"GetObjectsObj\". Did you mean \"nearObject\" or \"nearVector\"?","level":"error","msg":"unexpected error","query_type":"","time":"2024-03-22T06:05:17+03:00"}
2024-03-22 06:05:17,447 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 208


{
    "errors": [
        {
            "locations": [
                {
                    "column": 31,
                    "line": 1
                }
            ],
            "message": "Unknown argument \"nearText\" on field \"Inventory_data\" of type \"GetObjectsObj\". Did you mean \"nearObject\" or \"nearVector\"?",
            "path": null
        }
    ]
}


In [95]:
import weaviate
import json

client = weaviate.Client(
     embedded_options = EmbeddedOptions(),
    additional_headers = {
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")  # Replace with your inference API key
    }
)

2024-03-22 11:23:52,136 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:23:52,138 - DEBUG - http://localhost:8079 "GET /v1/.well-known/ready HTTP/1.1" 200 0
2024-03-22 11:23:52,142 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:23:52,144 - DEBUG - http://localhost:8079 "GET /v1/.well-known/openid-configuration HTTP/1.1" 404 0
2024-03-22 11:23:52,148 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-22 11:23:52,150 - DEBUG - http://localhost:8079 "GET /v1/meta HTTP/1.1" 200 871
2024-03-22 11:23:52,153 - DEBUG - Starting new HTTPS connection (1): pypi.org:443


embedded weaviate is already listening on port 8079


2024-03-22 11:23:52,722 - DEBUG - https://pypi.org:443 "GET /pypi/weaviate-client/json HTTP/1.1" 200 49836
/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [96]:
client.schema.delete_class("Question")

2024-03-22 11:23:55,590 - DEBUG - http://localhost:8079 "DELETE /v1/schema/Question HTTP/1.1" 200 0


In [97]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

{"level":"info","msg":"Created shard question_Gw7qkJGqLiBp in 1.570767ms","time":"2024-03-22T11:23:57+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T11:23:57+03:00","took":170525}
2024-03-22 11:23:57,920 - DEBUG - http://localhost:8079 "POST /v1/schema HTTP/1.1" 200 1062


In [98]:
import requests
import json
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

client.batch.configure(batch_size=100)  # Configure batch


2024-03-22 11:24:01,086 - DEBUG - Starting new HTTPS connection (1): raw.githubusercontent.com:443
2024-03-22 11:24:01,729 - DEBUG - https://raw.githubusercontent.com:443 "GET /weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json HTTP/1.1" 200 643


2024-03-22 11:24:01,741 - DEBUG - http://localhost:8079 "GET /v1/nodes HTTP/1.1" 200 212


In [99]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [100]:
with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(data[1:2]):  # Batch import data
        print(f"importing question: {i+1}")
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1


2024-03-22 11:24:12,897 - DEBUG - http://localhost:8079 "POST /v1/batch/objects HTTP/1.1" 200 None


In [101]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["biology"]})
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

2024-03-22 11:24:51,579 - DEBUG - Resetting dropped connection: localhost
2024-03-22 11:24:52,219 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 141


{
    "data": {
        "Get": {
            "Question": [
                {
                    "answer": "Elephant",
                    "category": "ANIMALS",
                    "question": "It's the only living mammal in the order Proboseidea"
                }
            ]
        }
    }
}


#### Connect to Weaviate

In [16]:
import weaviate
import json
from langchain.vectorstores import Weaviate

import os
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
     embedded_options = EmbeddedOptions(),
    additional_headers = {
        "X-Cohere-Api-Key": os.getenv("COHERE_APIKEY"),
        "X-HuggingFace-Api-Key": os.getenv("HUGGINGFACE_API_KEY")
    }
)

Started /home/babi/.cache/weaviate-embedded: process ID 2119023


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-03-22T12:59:14+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-03-22T12:59:14+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-03-22T12:59:14+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-03-22T12:59:14+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-03-22T12:59:14+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T12:59:15+03:00","took":512781}
{"level":"info","msg":"Completed loading shard langchain_dfad81fac8e04bf9a5dc95399e7c1293_mUFMm1hng3t7 in 1.158571ms","time":"2024-03-22T12:59:15+03:00"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T12:59:15+03:00","took":130709}
{"level":"info","msg":"Completed loading shard langchain_e0252e11ddd04080b46df73a9837af38_AbCQ6gA1xR0L in 729.266µs","time":"2024-03-22T12:59:15+03:00"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T12:59:15+03:00","took":89578}
{"level":"info","msg":"Completed loading shard langchain_e2e34564c8444b359990473fcc73b3b7_7sOdwDPs4XuR in 906.296µs","time":"2024-03-

#### Define a data collection

In [17]:
client.schema.delete_class("Inventory_data")

2024-03-22 12:59:20,647 - DEBUG - http://localhost:8079 "DELETE /v1/schema/Inventory_data HTTP/1.1" 200 0


In [127]:
# class_obj = {
#     "class": "Inventory_data",
#     "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
#     "moduleConfig": {
#         "text2vec-openai": {},
#         "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
#     }
# }

# client.schema.create_class(class_obj)

2024-03-22 11:51:13,952 - DEBUG - Resetting dropped connection: localhost
{"level":"info","msg":"Created shard inventory_data_nzoRXU73QDFL in 722.421µs","time":"2024-03-22T11:51:14+03:00"}
2024-03-22 11:51:14,105 - DEBUG - http://localhost:8079 "POST /v1/schema HTTP/1.1" 200 1068
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T11:51:14+03:00","took":138805}


In [18]:
class_obj = {
    "class": "Inventory_data",
    "vectorizer": "text2vec-huggingface",  # Use Hugging Face transformers
    "moduleConfig": {
        "text2vec-huggingface": {
          "model": "sentence-transformers/all-MiniLM-L6-v2",
           "options": {
            "waitForModel": "true",
            "useGPU": "true",
            "useCache": "true"
          }
        }
    }
}

client.schema.create_class(class_obj)

{"level":"info","msg":"Created shard inventory_data_rR4n7aFHzL4T in 1.105364ms","time":"2024-03-22T12:59:23+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-22T12:59:23+03:00","took":160667}
2024-03-22 12:59:23,863 - DEBUG - http://localhost:8079 "POST /v1/schema HTTP/1.1" 200 1088


####  Add objects

In [26]:
data = merged_df.iloc[:100, :].to_dict(orient='records')

In [27]:

client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, properties in enumerate(data):  # Batch import data
        batch.add_data_object(
            data_object=properties,
            class_name="Inventory_data"
        )

2024-03-22 13:00:23,697 - DEBUG - Resetting dropped connection: localhost
2024-03-22 13:00:23,708 - DEBUG - http://localhost:8079 "GET /v1/nodes HTTP/1.1" 200 212
2024-03-22 13:00:25,424 - DEBUG - http://localhost:8079 "POST /v1/batch/objects HTTP/1.1" 200 None


{'error': [{'message': 'update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}]}
{'error': [{'message': 'update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}]}
{'error': [{'message': 'update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}]}
{'error': [{'message': 'update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}]}
{'error': [{'message': 'update vector: failed with status: 429 error: Rate limit reached

In [25]:
data[0]

{'campaign_id': 'f4p5a01',
 'KPI': 'Engagement',
 'geo_targeting': 'All Country',
 'vertical': 'Sports',
 'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
 'game_key': 'b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed3176d99',
 'adformat': '320x480',
 'creative': 'Sensory Video',
 'renderingcontext': 'MobileOptimizedWeb',
 'osfamily': 'Android',
 'devicetype': 'Tablet',
 'devicemake': 'Samsung',
 'site_name': 'www.yahoo.com',
 'matchedfoldposition': 'Unknown',
 'browser': 'Chrome',
 'CTR': 0.0,
 'ER': 0.0}

#### Semantic search

In [14]:
response = (
    client.query
    .get("Inventory_data", ["campaign_id", "vertical", "targeting"])
    .with_near_text({"concepts": ["travel"]})
    .with_limit(10)
    .do()
)

2024-03-22 12:49:21,665 - DEBUG - Resetting dropped connection: localhost
2024-03-22 12:49:22,165 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 1288


In [15]:
response

{'data': {'Get': {'Inventory_data': [{'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I|